**To use this notebook:** Run one line at a time waiting for each cell to return before running the next cell.

## Simple Output

SimpleOutput is a widget that provides an output area to display all types of output. 

It is designed to minimise the size of messages and/or number of messages sent to the frontend. It is not supposed to be a drop in replacement for the Ipywidget `Output' widget, rather it provides an alternate type of interface.

Compared to the Ipywidgets `Output` maintains a synchronised model of all loaded outputs. Each item added to `SimpleOutput` is serialized and sent to the frontend.  There is no representation of the data left on the Python side meaning that `SimpleOutput` is more suitable for logging applications. 

## Methods

There are two methods to add outputs 
1. `push`
2. `set`

and one '`clear`' to clear the outputs.


### `push`

`push` serializes and sends data as a custom message which is appended to the existing output.

In [ ]:
import ipylab
from ipylab.simple_output import SimpleOutput

In [ ]:
so = SimpleOutput(layout={"max_height": "200px"})

In [ ]:
for i in range(50):
    so.push(f"test {i}\n")

In [ ]:
so

Or we could do it with one message...

In [ ]:
so.clear(wait=False)

In [ ]:
so.push(*(f"test {i}\n" for i in range(50)))

### Other formats are also supported

In [ ]:
from IPython.display import HTML, Markdown

so = SimpleOutput()
so.push(Markdown("## Markdown"), HTML("<h2>HTML</h2>"))
so

### set

`Set` is similar to push, but is run as task and clears the output prior at adding the new outputs. The task returns the number of outputs in use.

In [ ]:
t = so.set("Line one\n", "Line two")

In [ ]:
so

In [ ]:
assert so.length == t.result()  # noqa: S101
so.length

## max_continuous_streams

Notice that above the length is 1 even though we sent two values? 

This is because both items are streams, and by default they get put into the same output in the frontend. 

The maximum number of consecutive streams is configurable with `max_continuous_streams`.

In [ ]:
# Make each stream go into a new output.
so.max_continuous_streams = 0

In [ ]:
t = so.set("Line one\n", "Line two")

In [ ]:
assert so.length == t.result()  # noqa: S101
so.length

# AutoScroll

AutoScroll is a widget that provides automatic scrolling around a content widget. It is intended to be used in panels placed in the shell, and doesn't work correctly when used in notebooks.

**Note**

Autoscroll uses a relatively new feature `onscrollend` ([detail](https://developer.mozilla.org/en-US/docs/Web/API/Element/scrollend_event)) and **may not work well on Safari** for fast update rates.

## Ipylab log viewer

The Ipylab log viewer uses a `SimpleOutput` with an `AutoScroll` widget to scroll its output.

In [ ]:
app = ipylab.app
app.log_level = "DEBUG"

In [ ]:
for _ in range(10):
    app.log.debug("Debug")
    app.log.info("Info")
    app.log.warning("Warning")
    app.log.error("Error")
    app.log.exception("Exception")
    app.log.critical("Critical")

In [ ]:
app.log_viewer.connections[0].close()

## Example usage

In [ ]:
from datetime import datetime

import ipywidgets as ipw

import ipylab
from ipylab.simple_output import AutoScroll

In [ ]:
vb = ipw.VBox()
sw = AutoScroll(content=vb)
sw.sentinel_text = "sentinel"
sw_holder = ipw.VBox([sw], layout={"height": "200px", "border": "solid"})

enabled = ipw.Checkbox(description="Auto scroll", layout={"width": "120px"}, indent=False)
ipw.link((sw, "enabled"), (enabled, "value"))
sleep = ipw.FloatSlider(description="Sleep time (s)", value=0.3, min=0.05, max=1, step=0.01)

b_start = ipw.Button(description="Start", layout={"width": "max-content"})
b_clear = ipw.Button(description="Clear", layout={"width": "max-content"})
direction = ipw.RadioButtons(options=["fwd", "rev"], orientation="horizontal", layout={"width": "auto"})


def on_click(b):
    if b is b_start:
        if b.description == "Start":
            import asyncio

            async def generate_output():
                while True:
                    vb.children = (*vb.children, ipw.HTML(f"It is now {datetime.now().isoformat()}"))  # noqa: DTZ005
                    await asyncio.sleep(sleep.value)

            b.task = ipylab.app.to_task(generate_output())
            b.description = "Stop"
        else:
            b.task.cancel()
            b.description = "Start"
    if b is b_clear:
        vb.children = ()


b_start.on_click(on_click)
b_clear.on_click(on_click)


def _observe_direction(_):
    if direction.value == "fwd":
        sw.mode = "end"
        vb.layout.flex_flow = "column"
    else:
        sw.mode = "start"
        vb.layout.flex_flow = "column-reverse"


direction.observe(_observe_direction, "value")

p = ipylab.Panel(
    [ipw.HBox([enabled, sleep, direction, b_start, b_clear], layout={"justify_content": "center"}), sw_holder]
)
p.add_to_shell(mode=ipylab.InsertMode.split_right)